In [8]:
import time

from sentence_transformers import SentenceTransformer

from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

import plotly.express as px
import pandas as pd
import umap.umap_ as umap
# from umap import plot
# import umap.umap_.plot as umap
import numpy as np
import time
import itertools

In [9]:
# model = "microsoft/Multilingual-MiniLM-L12-H384"
# model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
# model = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"
model = "DeepPavlov/rubert-base-cased-sentence"
# model = "DeepPavlov/distilrubert-base-cased-conversational"
# model = "sentence-transformers/distiluse-base-multilingual-cased"

In [10]:
companies = pd.read_csv('../data/companies_large.csv', index_col=0)
companies.sample(5)

,Название,Описание,Источник,Brand-Image,Отрасль,Категория,Подкатегория,ИНН,ОГРН,КПП,Адрес,Телефоны,Почта,Сайт,Вконтакте,Год основания,Количество сотрудников,Название юридического лица,Дата регистрации,YouTube
10486,Дегтярский Литейно - Механический Завод,АО «Дегтярский Литейно - Механический Завод» (...,http://manufacturers.ru/company/degtyarskiy-li...,http://manufacturers.ru/sites/default/files/st...,Металлургия,Цветная металлургия,"Добыча, производство олова",6684018565,1156684000351,667101001.0,"г. Дегтярск, ул. Калинина 31Г",+7 (343) 328 08-28,dlmz_ao@mail.ru,http://dlmz.org,NaN,NaN,4,"Длмз, АО",03.02.2015,NaN
2539,Тубор,Компания Тубор - лидер производства качественн...,http://manufacturers.ru/company/tubor,http://manufacturers.ru/sites/default/files/st...,Транспортное машиностроение,Производство автокомплектующих,Производство автозапчастей,5246018014,1027739221290,774301001.0,"г. Москва, Новосходненское шоссе, дом 3а",+7-800-707-88-17 (розничная сеть) +7 (495) 231...,inbox@td.tubor.ru,http://tubor.ru,NaN,NaN,500,"Тубор, ООО",19.09.2002,NaN
1504,Сибирский сувенир,Компания WoodlandToys - это один из крупнейших...,http://manufacturers.ru/company/sibirskiy-suvenir,http://manufacturers.ru/sites/default/files/st...,Прочее производство,Производство прочих изделий и оборудования,Производство игрушек,5445037325,1145476091925,544501001.0,"633004, Новосибирская обл., г. Бердск, ул. Бар...",8(800) 550-11-93,info@woodlandtoys.ru,http://woodlandtoys.ru,NaN,NaN,67,"Сибирский Сувенир, ООО",24.07.2014,NaN
9813,ЛесМари,ТПК «ЛесМари» производит и продает: евровагонк...,http://manufacturers.ru/company/lesmari,http://manufacturers.ru/sites/default/files/st...,Лесная промышленность,"Лесозаготовительная промышленность, Деревообра...","Заготовка и переработка древесины, Производств...",NaN,NaN,NaN,"г. Йошкар-Ола, ул. Пугачева (заезд со стороны ...","8(8362)28-03-03 +7-987-701-03-03 (WhatsApp, Vi...",woodmari@mail.ru,http://evrovagonka-lipa.ru,NaN,NaN,NaN,NaN,NaN,NaN
1413,Кубанская лоза,ЗАО «Кубанская лоза» основано в 2004 г. на баз...,http://manufacturers.ru/company/kubanskaya-loza,http://manufacturers.ru/sites/default/files/st...,Пищевая промышленность,Производство алкогольной продукции,Винодельческая промышленность,2352036147,1042329064030,235201001.0,"353523, Краснодарский край, Темрюкский район, ...",+7(86148) 6-84-85 +7(86148) 6-84-85 +7(918) 02...,SECR@KLOZA.RU,http://xn--80aabzminj4b.xn--p1ai,NaN,NaN,72,"Кубанская Лоза, ЗАО",21.10.2004,NaN


In [11]:
sentence_embedder = SentenceTransformer(model, device='cuda:0')
sentence_embedder.max_seq_length = 512
sentence_embedder

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [12]:
# sentence_embedder = sentence_embedder.cpu()

In [13]:
# def chunks(iterable, n):
#     return [iterable[i:i+n] for i in range(0, len(iterable), n)]

In [14]:
descriptions_embs = sentence_embedder.encode(list(companies['Описание']), batch_size=32, show_progress_bar=True)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 5.80 GiB total capacity; 3.92 GiB already allocated; 38.44 MiB free; 3.95 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [9]:
descriptions_embs = descriptions_embs.astype(np.float16)
descriptions_embs.shape

(12600, 768)

In [11]:
umap_3d = umap.UMAP(n_components=3, n_neighbors=10, min_dist=0.1, init='random')

In [12]:
proj_3d = umap_3d.fit_transform(descriptions_embs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
categories = companies['Отрасль'].fillna('Null').apply(lambda x: x.split(',')[0])

In [15]:
fig = px.scatter_3d(x=proj_3d[:, 0], y=proj_3d[:, 1], z=proj_3d[:, 2], template='plotly_white', width=1400, height=800, color=categories)
fig.update_layout(showlegend=False)
fig.show(   )